In [17]:
import cv2.cv2 as cv2
import numpy as np
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
import random
import dataset
import os


In [18]:
def discriminator():
    model = Sequential()

    kernel_initializer = "random_normal"
    kernel_regularizer = None
    dropout_rate = 0.4
    pool_size = (2, 2)
    model.add(Conv2D(16, kernel_size=6, input_shape=(256, 256, 3), kernel_initializer=kernel_initializer,  kernel_regularizer=kernel_regularizer))
    model.add(LeakyReLU())
    model.add(MaxPool2D(pool_size=pool_size))
    model.add(Conv2D(8, kernel_size=4, kernel_initializer=kernel_initializer,  kernel_regularizer=kernel_regularizer))
    model.add(LeakyReLU())
    model.add(MaxPool2D(pool_size=pool_size))
    model.add(Conv2D(4, kernel_size=4, kernel_initializer=kernel_initializer,  kernel_regularizer=kernel_regularizer))
    model.add(LeakyReLU())
    model.add(MaxPool2D(pool_size=pool_size))
    model.add(Conv2D(1, kernel_size=4, kernel_initializer=kernel_initializer,  kernel_regularizer=kernel_regularizer))
    model.add(Flatten())
    model.add(LeakyReLU())
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation="sigmoid", kernel_initializer=kernel_initializer))
    ba = BinaryAccuracy(
        name="binary_accuracy", threshold=0.5
    )
    model.compile(Adam(), "binary_crossentropy", metrics=[ba])
    model.summary()

    return model

In [19]:
def get_randoms(batch=32, lower_bound=0, upper_bound=9999, ):
    rands = []
    for i in range(0, batch):
        rands.append(random.randint(lower_bound, upper_bound))
    return rands

def load_data(everyone_dir, me_dir, batch):
    indexes = get_randoms(batch=batch, lower_bound=0, upper_bound=1400)
    me = dataset.sort_names(os.listdir(me_dir))
    everyone = dataset.sort_names(os.listdir(everyone_dir))
    images_me = [me[indexes[i]] for i in range(len(indexes))]
    images_everyone = [everyone[indexes[i]] for i in range(len(indexes))]

    me_pictures, _ = dataset.load_face_pictures_list(me_dir, images_me, color_mode="rgb")
    everyone_pictures, _ = dataset.load_face_pictures_list(everyone_dir, images_everyone, color_mode="rgb")

    return me_pictures, everyone_pictures

In [21]:
#!g1.1
if __name__ == '__main__':
    model = discriminator()

    batch = 64

    me = np.ones(shape=(batch,))
    not_me = np.zeros(shape=(batch,))

    epochs = 4000

    for epoch in range(epochs):
        me_pictures, everyone_pictures = load_data(everyone_dir="./train_data/medical/CelebA-HQ-img-256-256",
                                                   me_dir="./me", batch=batch)

        me_pictures = me_pictures / 255
        everyone_pictures = everyone_pictures / 255

        features = np.concatenate((me_pictures, everyone_pictures))
        labels = np.concatenate((me, not_me))

        la = model.train_on_batch(features, labels)

        print("[Epoch %d/%d] [loss: %f, acc: %f]" % (
            epoch, epochs,
            la[0], la[1]))

        if epoch % 100 == 0:
            predicted = model.predict(features)
#             cv2.imshow("imgae", (features[12] * 255).astype('uint8'))
#             k = cv2.waitKey(30) & 0xff
#             if k == 27:
#                 break
            print("Real: " + str(labels[12]))
            print("Generated: " + str(predicted[12]))
            model.save("./saved_models/" + str(epoch) + "classifier", save_format="tf")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 251, 251, 16)      1744      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 251, 251, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 125, 125, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 8)       2056      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-c90e11c5ab67>", line 15, in <module>
    me_pictures = me_pictures / 255
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/kernel/lib/python3.7/site-packages/IPython/core/ultratb.py", line 316, in wr

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:811: UserWarning: The following variables cannot be serialized: model
  warnings.warn(message)


TypeError: object of type 'NoneType' has no len()

In [ ]:
#!g1.1
